In [ ]:
!pip install tabula-py pandas sentence-transformers faiss-cpu google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 64.9 MB/s eta 0:00:00


In [ ]:
import tabula
import numpy as np
import pandas as pd
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
from google.colab import userdata

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [ ]:
Gemini_API_Key = userdata.get("Gemini_API_Key")
if Gemini_API_Key is None:
 raise ValueError("No GEMINI_API_KEY found in Colab Secrets")
genai.configure(api_key=Gemini_API_Key)

In [ ]:
pdf_path = 'order.pdf'
tables = tabula.read_pdf(pdf_path, pages='all')
df = pd.concat(tables, ignore_index=True)
df

,order_id,order_status,customer,order_date,order_quantity,sales
0,3,Order\rFinished,Muhammed Mac\rIntyre,13/10/2010,6,523080
1,293,Order\rFinished,Barry French,1/10/2012,49,20246040
2,483,Order\rFinished,Clay Rozendal,10/7/2011,30,9931519
3,515,Order\rFinished,Carlos Soltero,28/8/2010,19,788540
4,613,Order\rFinished,Carl Jackson,17/6/2011,12,187080
5,643,Order\rFinished,Monica Federle,24/3/2011,21,5563640
6,678,Order\rReturned,Dorothy Badders,26/2/2010,44,456820
7,807,Order\rFinished,Neola Schneider,23/11/2010,45,393700
8,868,Order\rFinished,Carlos Daly,8/6/2012,32,1433680
9,933,Order\rFinished,Claudia Miner,4/8/2012,15,161220


In [ ]:
documents = []
for _, row in df.iterrows():
  doc = (
      f"Order ID: {row['order_id']} has status {row['order_status']}\n"
      f"Customer is {row['customer']}."
      f"Order Date is {row['order_date']}."
      f"Quantity Ordered is {row['order_quantity']}."
      f"Total Sales amount is {row['sales']}."
  )
  documents.append(doc)
print (documents)

['Order ID: 3 has status Order\rFinished\nCustomer is Muhammed Mac\rIntyre.Order Date is 13/10/2010.Quantity Ordered is 6.Total Sales amount is 523080.', 'Order ID: 293 has status Order\rFinished\nCustomer is Barry French.Order Date is 1/10/2012.Quantity Ordered is 49.Total Sales amount is 20246040.', 'Order ID: 483 has status Order\rFinished\nCustomer is Clay Rozendal.Order Date is 10/7/2011.Quantity Ordered is 30.Total Sales amount is 9931519.', 'Order ID: 515 has status Order\rFinished\nCustomer is Carlos Soltero.Order Date is 28/8/2010.Quantity Ordered is 19.Total Sales amount is 788540.', 'Order ID: 613 has status Order\rFinished\nCustomer is Carl Jackson.Order Date is 17/6/2011.Quantity Ordered is 12.Total Sales amount is 187080.', 'Order ID: 643 has status Order\rFinished\nCustomer is Monica Federle.Order Date is 24/3/2011.Quantity Ordered is 21.Total Sales amount is 5563640.', 'Order ID: 678 has status Order\rReturned\nCustomer is Dorothy Badders.Order Date is 26/2/2010.Quantit

In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedding_model.encode(documents, convert_to_numpy=True, show_progress_bar=True)
print(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[-0.04009406 -0.03813319  0.05240963 ... -0.08024698 -0.04358123
   0.05105152]
 [-0.03108115 -0.01880541 -0.00477832 ... -0.04830395 -0.05759401
  -0.0138682 ]
 [-0.06304455 -0.03100753  0.00952893 ... -0.0311431  -0.07441582
  -0.01718369]
 ...
 [ 0.02139986  0.00150407  0.07544634 ... -0.03270832 -0.01454126
  -0.02664858]
 [-0.09210812  0.01441542  0.04306166 ... -0.03566492 -0.00857275
   0.00213897]
 [-0.06764054 -0.00706395  0.00825971 ... -0.00477098 -0.08604837
   0.00569126]]


In [ ]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
faiss.write_index(index, 'faiss_index.faiss')


In [ ]:
def retrieve_context(query, k=3):
 query_embedding = embedding_model.encode([query])
 distances, indices = index.search(query_embedding, k)
 return "\n".join([documents[i] for i in  indices[0]])

In [ ]:
generation_config={
    'temperature':0.4,
    'max_output_tokens':512
}
print(generation_config)
gemini_model = genai.GenerativeModel(model_name='models/gemini-2.5-flash',generation_config=generation_config)

{'temperature': 0.4, 'max_output_tokens': 512}


In [ ]:
chat_history = []

def chat_with_bot(user_input):
  global chat_history

  context = retrieve_context(user_input)
  prompt = f"""
  You are a helpful conversational data analyst assistant. Please refer to the context below and answer the user's question.
  Context:
  {context}
  User's Question:
  {user_input}

  Rules:
  - Be conversational.
  - Answer only using the context.
  - If you don't know the answer,say you don't have enough information.
  """
  response = gemini_model.generate_content(prompt)
  answer = response.text
  chat_history.append({"user": user_input, "bot": answer})
  return answer

In [ ]:
print("Order Analytics Chat Bot Ready!!!")
print("Type 'exit' to stop\n")

while True:
  user_input = input("User: ")
  if user_input.lower() in ['exit', 'quit', 'bye']:
    print("Goodbye!")
    break
  response = chat_with_bot(user_input)
  print(f"Bot: {response}")
  print("-"*60)

Order Analytics Chat Bot Ready!!!
Type 'exit' to stop

User: what is total sales?
Bot: Based on the information provided, the total sales amount is 2,528,828.
------------------------------------------------------------
User: which customer has the highest sales?
Bot: Based on the information provided, Jim Radford has the highest sales with a total of 1,669,808.
------------------------------------------------------------
User: Bye
Goodbye!
